In [ ]:
from google.colab import drive 
drive.mount("/content/drive/")

In [ ]:
%cd ./drive/My Drive/Sound Classification
!ls

In [ ]:
!pip install numpy

In [ ]:
import librosa
import numpy as np

In [ ]:
lecture=np.load('lecture.npy',allow_pickle=True)
lecture.shape

In [ ]:
for i in range(len(lecture)):
  if lecture[i].shape!=(216,20):
    lecture = np.delete(lecture,i)


In [ ]:
lec=np.stack(lecture)

In [ ]:
lec.shape

In [ ]:
music=np.load('music_mfcc.npy',allow_pickle=True)
music.shape

In [ ]:
music1=np.load('guru1 (1).npy',allow_pickle=True)
music1.shape

In [ ]:
music=np.concatenate((music1,music),axis=0)

In [ ]:
silence=np.load('silence_mfcc.npy',allow_pickle=True)
silence.shape

In [ ]:
Y_v=np.append(np.zeros(len(lecture),dtype=int),np.ones(len(music),dtype=int))
Y_v=np.append(Y_v,2*np.ones(len(silence),dtype=int))
print(Y_v)
def oneHot(y, Ny):
    Y=np.zeros(Ny)
    Y[y]=1
    return Y
Ny = 3
Y_o = np.array([oneHot(j,Ny) for j in Y_v])
Y_o

In [ ]:
X=np.concatenate((lec,music,silence),axis=0)

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y_o, test_size=0.01) #test size 1%

In [ ]:
X_train.shape

CNN MODEL

In [ ]:
from numpy import asarray
from keras.models import Sequential,Input,Model
from keras.layers import Conv1D,Dense,MaxPool1D,Flatten,Dropout,Activation
x = Input(shape=(216,20))
y=Conv1D(216,(3),activation='relu',input_shape=(216,20))(x) #2^n
y=Dense(94, activation='relu')(y)
y=Conv1D(94,(3),activation='relu')(y)
y=Dense(24, activation='relu')(y)
#y=Conv1D(64,(3),activation='relu')(y)
#y=Conv1D(128,(3),activation='relu')(y)
y=MaxPool1D(2)(y)
y=Dropout(0.5)(y)
y=Flatten()(y)
y=Dense(16, activation='relu')(y)
y=Dense(3, activation='softmax')(y)
model_cnn = Model(inputs=x, outputs=y)
model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_cnn.summary()``

In [ ]:
history = model_cnn.fit(X_train, y_train,validation_split = 0.2, epochs=5) #train,validation,test

NN MODEL

In [ ]:
from numpy import asarray
from keras.models import Sequential
from keras.layers import Conv1D,Dense,MaxPool1D,Dropout,Flatten
model=Sequential()
model.add(Conv1D(16,(3),activation='relu',input_shape=(216,20)))
model.add(Conv1D(32,(3),activation='relu'))
model.add(Conv1D(64,(3),activation='relu'))
model.add(MaxPool1D(2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,validation_split = 0.2, epochs=5)

TEST MODEL

MFCC extraction for Test data

In [ ]:
'''data=[]
%cd ./project/lecture/chunks/
for x in range(76,220):
    #new_file = sound_file[x : x+5000]
    name="lecture_test"+ str(x) +".mp3"
    y, sr = librosa.load(name)
    arr=librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20).T
    if arr.shape==(216,20):
      data.append(arr)
data=np.array(data)'''

One hot Encodding for test model

In [ ]:
''''y=np.array([oneHot(j,Ny) for j in np.zeros(len(data),dtype=int)])
loss, accuracy = model.evaluate(data,y, verbose=0)  # Evaluate the model
print('Accuracy :%0.3f'%accuracy)
Y_pred = model.predict(data)
print(Y_pred[3][0])
for i in range(len(data)):
  if Y_pred[i][0]<0.5:
    print(i)
#Confusion Matrix
from sklearn.metrics import confusion_matrix
print( confusion_matrix(y.argmax(axis=1), Y_pred.argmax(axis=1)))''''

Data preprocessing for Testing


In [ ]:
x1=np.load('guru2.npy')
x2=np.load('kkkk.npy')
x3=np.load('guru3.npy')
x4=np.load('kbig1 (1).npy')
music_t=np.concatenate((x2,x4),axis=0)
lecture_t=np.concatenate((x1,x3),axis=0)

In [ ]:
data=np.concatenate((lecture_t,music_t),axis=0)
data.shape

In [ ]:
y=np.array([oneHot(j,Ny) for j in np.append(np.zeros(len(lecture_t),dtype=int),np.ones(len(music_t),dtype=int))])
loss, accuracy = model.evaluate(data,y, verbose=0)  # Evaluate the model
print('Accuracy :%0.3f'%accuracy)
Y_pred = model.predict(data)
#Confusion Matrix
from sklearn.metrics import confusion_matrix
print( confusion_matrix(y.argmax(axis=1), Y_pred.argmax(axis=1)))

In [ ]:
y=np.array([oneHot(j,Ny) for j in np.append(np.zeros(len(lecture_t),dtype=int),np.ones(len(music_t),dtype=int))])
loss, accuracy = model_cnn.evaluate(data,y, verbose=0)  # Evaluate the model
print('Accuracy :%0.3f'%accuracy)
Y_pred = model_cnn.predict(data)
#Confusion Matrix
from sklearn.metrics import confusion_matrix
print( confusion_matrix(y.argmax(axis=1), Y_pred.argmax(axis=1)))